<a href="https://colab.research.google.com/github/reeshabhkumarranjan/ML-Course-Project/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import numpy as np
import glob
from keras import Sequential
import sklearn, sklearn.model_selection
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Activation
from keras.utils import to_categorical, plot_model

In [18]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

# loading data
colab_directory = '/content/drive/My Drive/Colab Notebooks/ML Course Project/'
x = np.load(colab_directory + "x.npy")
y = np.load(colab_directory + "y.npy")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# common variables
epochs = 5
showWorking = False
showWorkingAverage = True
batchSize = 10
dropoutRate = 0.5
activationFunction = 'relu'
iterations = 10

In [20]:
# LSTM model
accuracy_average = 0
for i in range(iterations):
	model_lstm = Sequential([
		LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2])),
		# Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax'),
	])
 
	# save the architecture of the model
	plot_model(model_lstm, colab_directory + 'model_lstm.png')

	model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model_lstm.fit(x_train, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
	accuracy_average += model_lstm.evaluate(x_test, y_test, batch_size=batchSize, verbose=showWorking)[1]
	if showWorkingAverage:
		print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
accuracy_average /= iterations
print("Accuracy of LSTM model on test: ", accuracy_average)
print()

1: 0.7899999976158142
2: 0.8349999964237214
3: 0.7533333331346513
4: 0.7924999974668027
5: 0.7899999970197678
6: 0.8099999949336052
7: 0.8085714250802994
8: 0.8187499959021807
9: 0.8277777728107241
10: 0.8389999946951866
Accuracy of LSTM model on test:  0.8389999946951866



In [21]:
# CNN-LSTM
accuracy_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 20, x_test.shape[2]))

	conv1 = Conv1D(filters=64, kernel_size=3, activation=activationFunction)
	conv2 = Conv1D(filters=64, kernel_size=3, activation=activationFunction)
	model_lstm_cnn = Sequential([
		TimeDistributed(conv1, input_shape=(None, 20, x_train.shape[2])),
		TimeDistributed(conv2),
		# TimeDistributed(Dropout(dropoutRate)),
		TimeDistributed(MaxPooling1D(pool_size=2)),
		TimeDistributed(Flatten()),
		LSTM(200),
		# Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax')
	])
 
	# save the architecture of the model
	plot_model(model_lstm_cnn, colab_directory + 'model_lstm_cnn.png')

	model_lstm_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model_lstm_cnn.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)

	accuracy_average += model_lstm_cnn.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
	if showWorkingAverage:
		print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
accuracy_average /= iterations
print("Accuracy of LSTM-CNN model on test: ", accuracy_average)
print()

1: 0.9499999940395355
2: 0.9299999922513962
3: 0.9266666571299235
4: 0.92999999076128
5: 0.9299999916553497
6: 0.9266666591167451
7: 0.9271428491388048
8: 0.9262499921023846
9: 0.9266666577921973
10: 0.9259999907016754
Accuracy of LSTM-CNN model on test:  0.9259999907016754



In [22]:
# LSTM + conv
accuracy_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 1, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 1, 20, x_test.shape[2]))

	model_lstm_conv = Sequential([
		ConvLSTM2D(filters=64, kernel_size=(1,3), input_shape=(3, 1, 20, x_train.shape[2])),
		Activation(activationFunction),
		# Dropout(dropoutRate),
		Flatten(),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax')
	])
 
	# save the architecture of the model
	plot_model(model_lstm_conv, colab_directory + 'model_lstm_conv.png')
 
	model_lstm_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model_lstm_conv.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)

	accuracy_average += model_lstm_conv.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
	if showWorkingAverage:
		print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
accuracy_average /= iterations
print("Accuracy of LSTM-Conv model on test: ", accuracy_average)
print()

1: 0.8799999952316284
2: 0.8999999910593033
3: 0.9066666583220164
4: 0.9149999901652337
5: 0.919999990463257
6: 0.9183333228031795
7: 0.9214285612106324
8: 0.9224999897181989
9: 0.9022222134802078
10: 0.905999991297722
Accuracy of LSTM-Conv model on test:  0.905999991297722

